In [ ]:
from all_imports import *
import requests
import csv
import _modules 

In [ ]:
#TODO 

# find first, all studies regarind outdoor thermal comfort
# check the primary pattern
# second run, parse thorugh the filtered list of studies and get the metadata
# filter the data by needing to have all valid data, exepct number of votes or thermal history

# OLd

In [ ]:
import requests
import csv
import re
from xml.etree import ElementTree as ET

# Configuration
API_KEY = _modules.SCOPUS_API_KEY  # Replace with your Scopus API key
INST_TOKEN = _modules.INST_TOKEN   # Replace with your institutional token
SEARCH_QUERY = 'TITLE-ABS-KEY("outdoor thermal comfort study") AND PUBYEAR AFT 1999 AND PUBYEAR BEF 2025'
BASE_URL = 'https://api.elsevier.com/content/search/scopus'
CSV_FILENAME = 'scopus_outdoor_thermal_comfort.csv'

# Define regex patterns for extracting locations
location_patterns = [
    r'\b(?:located in|in the region of|in the city of|conducted in|performed in|studied in)\s+([A-Z][a-zA-Z]+(?:\s[A-Z][a-zA-Z]+)*)',
    r'\b(?:country of|nation of|state of)\s+([A-Z][a-zA-Z]+(?:\s[A-Z][a-zA-Z]+)*)',
]

# Define regex patterns for extracting number of participants and votes
participants_pattern = r'\b(?:number of participants|sample size|participants? count|sample size is)\s*[:\-]?\s*(\d+)'
votes_pattern = r'\b(?:number of votes|votes count|total votes|voted by)\s*[:\-]?\s*(\d+)'

# Enhanced primary regex pattern to identify studies on outdoor thermal comfort with participants
primary_pattern = r'(?=.*\bstudy\b)(?=.*\boutdoor\s+comfort\b)(?=.*\bparticipants?\b)'

# Define secondary regex patterns for additional keywords related to outdoor thermal comfort
secondary_patterns = [
    r'\bthermal\s+comfort\b',
    r'\boutdoor\s+(environment|settings|conditions|spaces|areas)\b',
    r'\bmicroclimate\b',
    r'\burban\s+heat\s+island\b',
    r'\bheat\s+stress\b',
    r'\bthermal\s+perception\b',
    r'\bphysiological\s+responses?\b',
    r'\bcomfort\s+models?\b',
    r'\beffect\s+of\s+.+\bon\s+outdoor\s+comfort\b',
    r'\bassessment\s+of\s+thermal\s+comfort\s+in\b',
    r'\bfield\s+measurements?\b',
    r'\bsimulation(s)?\b',
    r'\bquestionnaire(s)?\b',
    r'\bresearch\s+(aim|objective|purpose|methodology|findings|results)\b',
]

# Function to extract city, country, number of participants, and number of votes using regex patterns
def extract_details(text):
    found_locations = []
    matched_sentences = []
    number_of_participants = None
    number_of_votes = None
    text_references = []
    
    sentences = re.split(r'(?<=[.!?])\s+', text)  # Split text into sentences

    for sentence in sentences:
        for pattern in location_patterns:
            matches = re.findall(pattern, sentence)
            if matches:
                found_locations.extend(matches)
                matched_sentences.append(sentence)
        
        # Check for number of participants
        participants_match = re.search(participants_pattern, sentence, re.IGNORECASE)
        if participants_match:
            number_of_participants = participants_match.group(1)
            text_references.append(f"Participants data found: {sentence}")

        # Check for number of votes
        votes_match = re.search(votes_pattern, sentence, re.IGNORECASE)
        if votes_match:
            number_of_votes = votes_match.group(1)
            text_references.append(f"Votes data found: {sentence}")

    # Remove duplicate locations
    found_locations = list(set(found_locations))
    
    return found_locations, number_of_participants, number_of_votes, matched_sentences, text_references

# Function to determine if text indicates a study on outdoor thermal comfort with participants
def is_outdoor_thermal_comfort_study(text):
    if re.search(primary_pattern, text, re.IGNORECASE):
        for pattern in secondary_patterns:
            if re.search(pattern, text, re.IGNORECASE):
                return True
    return False

# Function to fetch full-text document from Elsevier API
def fetch_full_text(doi):
    url = f'https://api.elsevier.com/content/article/doi/{doi}?view=FULL'
    headers = {
        'X-ELS-APIKey': API_KEY,
        'X-ELS-Insttoken': INST_TOKEN,
        'Accept': 'text/plain'  # Change to 'text/xml' for XML format
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        return response.text, False  # Return False for 'Error' column if no error
    else:
        print(f"Error fetching full text for DOI {doi}: {response.status_code}, {response.text}")
        return None, True  # Return True for 'Error' column if error

# Function to fetch metadata from Scopus API
def fetch_metadata(api_key, inst_token, query, max_papers=10):
    headers = {
        'X-ELS-APIKey': api_key,
        'X-ELS-Insttoken': inst_token,
        'Accept': 'application/json'
    }
    
    params = {
        'query': query,
        'view': 'COMPLETE',
        'count': max_papers
    }

    papers = []
    start_index = 0
    
    while len(papers) < max_papers:
        params['start'] = start_index
        response = requests.get(BASE_URL, headers=headers, params=params)

        if response.status_code != 200:
            print(f"Error fetching data: {response.status_code}, {response.text}")
            break

        data = response.json()

        if 'search-results' not in data or 'entry' not in data['search-results']:
            break

        entries = data['search-results']['entry']
        if not entries:
            break

        for entry in entries:
            if len(papers) >= max_papers:
                break
            
            doi = entry.get('prism:doi', 'N/A')
            title = entry.get('dc:title', 'N/A')
            authors = entry.get('dc:creator', 'N/A')
            publication_date = entry.get('prism:coverDate', 'N/A')
            volume = entry.get('prism:volume', 'N/A')
            issue = entry.get('prism:issueIdentifier', 'N/A')
            keywords = entry.get('authkeywords', 'N/A')
            document_type = entry.get('subtypeDescription', 'N/A')

            # Fetch full text for the article
            full_text, error = fetch_full_text(doi)
            if full_text:
                # Determine if the full text indicates an outdoor thermal comfort study with participants
                if is_outdoor_thermal_comfort_study(full_text):
                    # Extract locations, number of participants, votes, matched sentences, and text references from full text
                    cities, num_participants, num_votes, matched_sentences, text_references = extract_details(full_text)
                else:
                    continue  # Skip if the text doesn't match the thermal comfort criteria
            else:
                cities, num_participants, num_votes, matched_sentences, text_references = [], None, None, [], []

            if isinstance(keywords, list):
                keywords = '; '.join(keywords)

            papers.append({
                'DOI': doi,
                'Title': title,
                'Authors': authors,
                'Publication Date': publication_date,
                'Volume': volume,
                'Issue': issue,
                'Keywords': keywords,
                'Document Type': document_type,
                'City': '; '.join(cities) if cities else 'N/A',
                'Country': '; '.join(cities) if cities else 'N/A',
                'Number of Participants': num_participants if num_participants else 'N/A',
                'Number of Votes': num_votes if num_votes else 'N/A',
                'Matched Sentences': ' | '.join(matched_sentences) if matched_sentences else 'N/A',
                'Text Reference': ' | '.join(text_references) if text_references else 'N/A',
                'Error': 'True' if error else ''  # Add 'Error' column data
            })

        start_index += len(entries)
        
    return papers

# Function to save papers to CSV
def save_to_csv(filename, papers):
    fieldnames = ['DOI', 'Title', 'Authors', 'Publication Date', 'Volume', 'Issue', 'Keywords', 'Document Type', 'City', 'Country', 'Number of Participants', 'Number of Votes', 'Matched Sentences', 'Text Reference', 'Error']
    
    with open(filename, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        for paper in papers:
            writer.writerow(paper)
    print(f"Data saved to {filename}")

# Main script
if __name__ == "__main__":
    papers = fetch_metadata(API_KEY, INST_TOKEN, SEARCH_QUERY, max_papers=5)  # Limit to 10 papers
    save_to_csv(CSV_FILENAME, papers)
